In [59]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt/'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import re
import scipy.stats as stats

In [50]:
def summarize_muts_per_impact_gene(maf):
    maf = maf[maf['oncogenic'].notnull()]
    ncases = len(set(maf['Tumor_Sample_Barcode']))
    d = {}
    impactGenes = set(['ABL1', 'ACVR1', 'AGO2', 'AKT1', 'AKT2', 'AKT3', 'ALK', 'ALOX12B', 'ANKRD11', 'APC', 'AR', 'ARAF', 'ARID1A', 'ARID1B', 'ARID2', 'ARID5B', 'ASXL1', 'ASXL2', 'ATM', 'ATR', 'ATRX', 'AURKA', 'AURKB', 'AXIN1', 'AXIN2', 'AXL', 'B2M', 'BABAM1', 'BAP1', 'BARD1', 'BBC3', 'BCL10', 'BCL2', 'BCL2L1', 'BCL2L11', 'BCL6', 'BCOR', 'BIRC3', 'BLM', 'BMPR1A', 'BRAF', 'BRCA1', 'BRCA2', 'BRD4', 'BRIP1', 'BTK', 'CALR', 'CARD11', 'CARM1', 'CASP8', 'CBFB', 'CBL', 'CCND1', 'CCND2', 'CCND3', 'CCNE1', 'CD274', 'CD276', 'CD79A', 'CD79B', 'CDC42', 'CDC73', 'CDH1', 'CDK12', 'CDK4', 'CDK6', 'CDK8', 'CDKN1A', 'CDKN1B', 'CDKN2A', 'CDKN2B', 'CDKN2C', 'CEBPA', 'CENPA', 'CHEK1', 'CHEK2', 'CIC', 'CREBBP', 'CRKL', 'CRLF2', 'CSDE1', 'CSF1R', 'CSF3R', 'CTCF', 'CTLA4', 'CTNNB1', 'CUL3', 'CXCR4', 'CYLD', 'CYSLTR2', 'DAXX', 'DCUN1D1', 'DDR2', 'DICER1', 'DIS3', 'DNAJB1', 'DNMT1', 'DNMT3A', 'DNMT3B', 'DOT1L', 'DROSHA', 'DUSP4', 'E2F3', 'EED', 'EGFL7', 'EGFR', 'EIF1AX', 'EIF4A2', 'EIF4E', 'ELF3', 'EP300', 'EPAS1', 'EPCAM', 'EPHA3', 'EPHA5', 'EPHA7', 'EPHB1', 'ERBB2', 'ERBB3', 'ERBB4', 'ERCC2', 'ERCC3', 'ERCC4', 'ERCC5', 'ERF', 'ERG', 'ERRFI1', 'ESR1', 'ETV1', 'ETV6', 'EZH1', 'EZH2', 'FAM123B', 'FAM175A', 'FAM46C', 'FAM58A', 'FANCA', 'FANCC', 'FAT1', 'FBXW7', 'FGF19', 'FGF3', 'FGF4', 'FGFR1', 'FGFR2', 'FGFR3', 'FGFR4', 'FH', 'FLCN', 'FLT1', 'FLT3', 'FLT4', 'FOXA1', 'FOXL2', 'FOXO1', 'FOXP1', 'FUBP1', 'FYN', 'GATA1', 'GATA2', 'GATA3', 'GLI1', 'GNA11', 'GNAQ', 'GNAS', 'GPS2', 'GREM1', 'GRIN2A', 'GSK3B', 'H3F3A', 'H3F3B', 'H3F3C', 'HGF', 'HIST1H1C', 'HIST1H2BD', 'HIST1H3A', 'HIST1H3B', 'HIST1H3C', 'HIST1H3D', 'HIST1H3E', 'HIST1H3F', 'HIST1H3G', 'HIST1H3H', 'HIST1H3I', 'HIST1H3J', 'HIST2H3C', 'HIST2H3D', 'HIST3H3', 'HLA-A', 'HLA-B', 'HNF1A', 'HOXB13', 'HRAS', 'ICOSLG', 'ID3', 'IDH1', 'IDH2', 'IFNGR1', 'IGF1', 'IGF1R', 'IGF2', 'IKBKE', 'IKZF1', 'IL10', 'IL7R', 'INHA', 'INHBA', 'INPP4A', 'INPP4B', 'INPPL1', 'INSR', 'IRF4', 'IRS1', 'IRS2', 'JAK1', 'JAK2', 'JAK3', 'JUN', 'KDM5A', 'KDM5C', 'KDM6A', 'KDR', 'KEAP1', 'KIT', 'KLF4', 'KMT2B', 'KMT5A', 'KNSTRN', 'KRAS', 'LATS1', 'LATS2', 'LMO1', 'LYN', 'MALT1', 'MAP2K1', 'MAP2K2', 'MAP2K4', 'MAP3K1', 'MAP3K13', 'MAP3K14', 'MAPK1', 'MAPK3', 'MAPKAP1', 'MAX', 'MCL1', 'MDC1', 'MDM2', 'MDM4', 'MED12', 'MEF2B', 'MEN1', 'MET', 'MGA', 'MITF', 'MLH1', 'KMT2A', 'KMT2B', 'KMT2C', 'MPL', 'MRE11A', 'MSH2', 'MSH3', 'MSH6', 'MSI1', 'MSI2', 'MST1', 'MST1R', 'MTOR', 'MUTYH', 'MYC', 'MYCL1', 'MYCN', 'MYD88', 'MYOD1', 'NBN', 'NCOA3', 'NCOR1', 'NEGR1', 'NF1', 'NF2', 'NFE2L2', 'NFKBIA', 'NKX2-1', 'NKX3-1', 'NOTCH1', 'NOTCH2', 'NOTCH3', 'NOTCH4', 'NPM1', 'NRAS', 'NSD1', 'NTHL1', 'NTRK1', 'NTRK2', 'NTRK3', 'NUF2', 'NUP93', 'PAK1', 'PAK7', 'PALB2', 'PARK2', 'PARP1', 'PAX5', 'PBRM1', 'PDCD1', 'PDCD1LG2', 'PDGFRA', 'PDGFRB', 'PDPK1', 'PGR', 'PHOX2B', 'PIK3C2G', 'PIK3C3', 'PIK3CA', 'PIK3CB', 'PIK3CD', 'PIK3CG', 'PIK3R1', 'PIK3R2', 'PIK3R3', 'PIM1', 'PLCG2', 'PLK2', 'PMAIP1', 'PMS1', 'PMS2', 'PNRC1', 'POLD1', 'POLE', 'PPARG', 'PPM1D', 'PPP2R1A', 'PPP4R2', 'PPP6C', 'PRDM1', 'PRDM14', 'PREX2', 'PRKAR1A', 'PRKCI', 'PRKD1', 'PTCH1', 'PTEN', 'PTP4A1', 'PTPN11', 'PTPRD', 'PTPRS', 'PTPRT', 'RAB35', 'RAC1', 'RAC2', 'RAD21', 'RAD50', 'RAD51', 'RAD51C', 'RAD51L1', 'RAD51L3', 'RAD52', 'RAD54L', 'RAF1', 'RARA', 'RASA1', 'RB1', 'RBM10', 'RECQL', 'RECQL4', 'REL', 'RET', 'RFWD2', 'RHEB', 'RHOA', 'RICTOR', 'RIT1', 'RNF43', 'ROS1', 'RPS6KA4', 'RPS6KB2', 'RPTOR', 'RRAGC', 'RRAS', 'RRAS2', 'RTEL1', 'RUNX1', 'RXRA', 'RYBP', 'SDHA', 'SDHAF2', 'SDHB', 'SDHC', 'SDHD', 'SESN1', 'SESN2', 'SESN3', 'SETD2', 'SF3B1', 'SH2B3', 'SH2D1A', 'SHOC2', 'SHQ1', 'SLX4', 'SMAD2', 'SMAD3', 'SMAD4', 'SMARCA4', 'SMARCB1', 'SMARCD1', 'SMO', 'SMYD3', 'SOCS1', 'SOS1', 'SOX17', 'SOX2', 'SOX9', 'SPEN', 'SPOP', 'SPRED1', 'SRC', 'SRSF2', 'STAG2', 'STAT3', 'STAT5A', 'STAT5B', 'STK11', 'STK19', 'STK40', 'SUFU', 'SUZ12', 'SYK', 'TAP1', 'TAP2', 'TBX3', 'TCEB1', 'TCF3', 'TCF7L2', 'TEK', 'TERT', 'TET1', 'TET2', 'TGFBR1', 'TGFBR2', 'TMEM127', 'TMPRSS2', 'TNFAIP3', 'TNFRSF14', 'TOP1', 'TP53', 'TP53BP1', 'TP63', 'TRAF2', 'TRAF7', 'TSC1', 'TSC2', 'TSHR', 'U2AF1', 'UPF1', 'VEGFA', 'VHL', 'VTCN1', 'WHSC1', 'WHSC1L1', 'WT1', 'WWTR1', 'XIAP', 'XPO1', 'XRCC2', 'YAP1', 'YES1', 'ZFHX3', 'ZRSR2'])
    for gene in impactGenes:
        geneMaf = maf[maf['Hugo_Symbol'] == gene]
        nmut = geneMaf.drop_duplicates(subset=['Tumor_Sample_Barcode']).shape[0]
        d[gene] = (nmut, ncases - nmut)
    return d
    

In [128]:
def get_gene_mutation_freqs_for_tcga(filePath, nLines, sigFilePath):
    dfCancerType = analysis_utils.load_in_df_with_progress(filePath = filePath, nLinesFile = nLines)
    sigCancerType = pd.read_table(sigFilePath)
    sigCancerType['Nmut_Mb'] = sigCancerType['Number of Mutations'].apply(lambda x: x/30.0)
    cancerTypeHypermutators = sigCancerType[sigCancerType['Nmut_Mb'] > 30]['Sample Name']
    cancerTypeHypermutatorMaf = dfCancerType[dfCancerType['Tumor_Sample_Barcode'].isin(cancerTypeHypermutators)]
    cancerTypeHypermutatorMafOnc = cancerTypeHypermutatorMaf[cancerTypeHypermutatorMaf['oncogenic'].notnull()]
    
    summary = summarize_muts_per_impact_gene(cancerTypeHypermutatorMafOnc)
    return summary, cancerTypeHypermutatorMafOnc
    

In [67]:
def do_fisher_test_impact_vs_exome(impactCounts, exomeCounts):
    listOfDicts = []
    for gene in impactCounts.keys():
        nImpactGene, nImpactNotGene = impactCounts[gene]
        nExomeGene, nExomeNotGene = exomeCounts[gene]
        oddsRatio, pValue = stats.fisher_exact([[nImpactGene, nImpactNotGene],
                                                                    [nExomeGene, nExomeNotGene]])
        listOfDicts.append({
            'Hugo_Symbol': gene, 'oddsRatio': oddsRatio, 'pVal': pValue
        })
    return pd.DataFrame(listOfDicts)
        
    

In [94]:
def prepare_summary_comparisson_impact_v_exome(summaryTCGA, summaryIMPACT):
    df = do_fisher_test_impact_vs_exome(summaryIMPACT, summaryTCGA)
    geneSizeDict = analysis_utils.get_cds_size_targeted_by_impact(infoFilePath = pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/impact_gene_reference_signatures.tsv')
    df['geneSize'] = df['Hugo_Symbol'].apply(lambda x: geneSizeDict[x] if x in geneSizeDict else None)
    df['displayName'] = df.apply(lambda row: row['Hugo_Symbol'] if row['pVal'] < 1e-4 else None, axis=1)
    return df


In [129]:
endoTCGASummary, endoOncMaf = get_gene_mutation_freqs_for_tcga(filePath = pathPrefix + '/ifs/res/taylorlab/jonssonp/msk_impact_gml_som/tcga/mutsig/UCEC.mc3.v0.2.8.PUBLIC.maf.2018-01-29.oncokb.vep.trinuc.maf',
                                      nLines = 945103, sigFilePath = pathPrefix + '/ifs/res/taylorlab/jonssonp/msk_impact_gml_som/tcga/mutsig/UCEC.mc3.v0.2.8.PUBLIC.maf.2018-01-29.oncokb.vep.mutsig.txt')

coloTCGASummary, coloOncMaf = get_gene_mutation_freqs_for_tcga(filePath = pathPrefix + '/ifs/res/taylorlab/jonssonp/msk_impact_gml_som/tcga/mutsig/COAD.mc3.v0.2.8.PUBLIC.maf.2018-01-29.oncokb.vep.trinuc.maf',
                                      nLines = 277114, sigFilePath = pathPrefix + '/ifs/res/taylorlab/jonssonp/msk_impact_gml_som/tcga/mutsig/COAD.mc3.v0.2.8.PUBLIC.maf.2018-01-29.oncokb.vep.mutsig.txt')


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (4,138,145,146,148) have mixed types. Specify dtype option on import or set low_memory=False.
  


1.06 percent done


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (4,146,148) have mixed types. Specify dtype option on import or set low_memory=False.
  


2.12 percent done


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (76,138,146,148) have mixed types. Specify dtype option on import or set low_memory=False.
  


...

93.82 percent done


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (76,138) have mixed types. Specify dtype option on import or set low_memory=False.
  """


97.43 percent done


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (146) have mixed types. Specify dtype option on import or set low_memory=False.
  """


101.04 percent done
performing big concat then returning


In [23]:
endometrialIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'all')
endometrialHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'Hypermutated')

colorectalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'all')
colorectalHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'Hypermutated')

gliomaIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'all')
gliomaHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'Hypermutated')


In [ ]:
allImpactMuts = analysis_utils.load_in_df_with_progress(filePath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf', nLinesFile = 275000)

In [78]:
allImpactMuts = maf_analysis_utils.fix_mll_genes(allImpactMuts)

In [79]:
endometrialHypermutators = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(endometrialHyperIds)]
colorectalHypermutators = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(colorectalHyperIds)]
gliomaHypermutators = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(gliomaHyperIds)]

In [118]:
summaryEndometrialImpact = summarize_muts_per_impact_gene(endometrialHypermutators)

In [98]:
summaryColorectalImpact = summarize_muts_per_impact_gene(colorectalHypermutators)

In [123]:
#dfColo = prepare_summary_comparisson_impact_v_exome(coloTCGASummary, summaryColorectalImpact)
dfEndo = prepare_summary_comparisson_impact_v_exome(endoTCGASummary, summaryEndometrialImpact)

In [124]:
dfColo.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/tcgaVsImpactGenesMutated_colon.tsv', index=False, sep='\t')
dfEndo.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/tcgaVsImpactGenesMutated_endo.tsv', index=False, sep='\t')



In [130]:
#def summarize_onc_mut_freqs(maf, sigs);
    

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,...,oncogenic,LEVEL_1,LEVEL_2A,LEVEL_2B,LEVEL_3A,LEVEL_3B,LEVEL_4,LEVEL_R1,Highest_level,Ref_Tri
766,APC,324,.,GRCh37,5,112173917,112173917,+,Nonsense_Mutation,SNP,...,Likely Oncogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCG
767,KRAS,3845,.,GRCh37,12,25398284,25398284,+,Missense_Mutation,SNP,...,Oncogenic,NaN,NaN,NaN,NaN,NaN,"LY3214996,GDC-0994,KO-947","Cetuximab,Panitumumab",LEVEL_4,ACC
1091,KMT2D,8085,.,GRCh37,12,49420871,49420871,+,Nonsense_Mutation,SNP,...,Likely Oncogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCG
1420,CREBBP,1387,.,GRCh37,16,3778420,3778420,+,Nonsense_Mutation,SNP,...,Likely Oncogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCA
1849,PTPRS,5802,.,GRCh37,19,5214592,5214592,+,Missense_Mutation,SNP,...,Predicted Oncogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCG
2729,PIK3R1,5295,.,GRCh37,5,67591106,67591106,+,Missense_Mutation,SNP,...,Predicted Oncogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TTA
3266,RBM10,8241,.,GRCh37,X,47032551,47032551,+,Missense_Mutation,SNP,...,Predicted Oncogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCG
6013,KMT2D,8085,.,GRCh37,12,49426631,49426631,+,Nonsense_Mutation,SNP,...,Likely Oncogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACA
6378,CTCF,10664,.,GRCh37,16,67654642,67654642,+,Missense_Mutation,SNP,...,Likely Oncogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCG
6420,NCOR1,9611,.,GRCh37,17,15968334,15968334,+,Splice_Site,SNP,...,Likely Oncogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCT
